In [1]:
# Required Libraries
import numpy as np
import pandas as pd

# Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Misc Libraries
import warnings
warnings.filterwarnings('ignore')

2023-02-02 16:58:58.215705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 16:58:58.331314: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-02 16:58:58.334398: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-02 16:58:58.334412: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
# Global Variables
TRAIN_PATH = '../data/train/images'
TEST_PATH = '../data/test/images'
VAL_PATH = '../data/valid/images'
TRAIN_FEATURE = '../data/train/keypoints.csv'
TEST_FEATURE = '../data/test/keypoints.csv'
VAL_FEATURE = '../data/valid/keypoints.csv'
SCALING_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_KEYPOINTS = 34

In [3]:
# read target feature for each dataset
y_train = pd.read_csv(TRAIN_FEATURE)
y_test = pd.read_csv(TEST_FEATURE)
y_val = pd.read_csv(VAL_FEATURE)

y_train.head()

,imageId,joint1x,joint1y,joint2x,joint2y,joint3x,joint3y,joint4x,joint4y,joint5x,...,joint13x,joint13y,joint14x,joint14y,joint15x,joint15y,joint16x,joint16y,joint17x,joint17y
0,162346,57.050000,97.533333,49.700000,90.533333,0.000000,0.000000,30.450000,120.866667,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,53131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,41859,49.350000,95.656652,50.400000,92.291845,48.650000,93.253219,54.250000,90.369099,0.000000,...,56.700000,144.206009,54.250000,167.27897,58.100000,171.124464,73.150000,179.776824,61.600000,199.004292
3,402172,114.800000,45.114754,118.300000,43.016393,113.400000,43.016393,122.500000,46.688525,0.000000,...,116.900000,114.360656,128.100000,158.95082,114.100000,150.032787,131.950000,177.836066,111.300000,195.672131
4,62531,139.722772,53.900000,143.271287,51.100000,136.174257,51.100000,147.263366,52.850000,131.738614,...,128.633663,115.500000,148.594059,143.50000,126.415842,143.150000,149.924752,172.550000,124.641584,172.550000


In [4]:
# image from directory set up pipeline
train = ImageDataGenerator(
    rescale = 1/255,
)

val = ImageDataGenerator(
    rescale = 1.0/255,
)

test = ImageDataGenerator(
    rescale = 1.0/255,
)

In [5]:
train_images = train.flow_from_directory(
    '../data/train/',
    batch_size = BATCH_SIZE,
    class_mode = None,
)

val_images = val.flow_from_directory(
    '../data/valid',
    batch_size = BATCH_SIZE,
    class_mode = None,
)

test_images = test.flow_from_directory(
    '../data/test',
    batch_size = BATCH_SIZE,
    class_mode = None,
)

Found 19865 images belonging to 1 classes.
Found 1045 images belonging to 1 classes.
Found 4967 images belonging to 1 classes.


In [6]:
# build model
model = tf.keras.Sequential()
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', input_shape = (SCALING_SIZE[0], SCALING_SIZE[1], 1)))
for i in range(49):
    if(i < 24):
        model.add(layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
    else:
        model.add(layers.Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))

model.add(layers.MaxPooling2D(pool_size = (2,2)))
model.add(layers.MaxPooling2D(pool_size = (2,2)))
for i in range(10):
    if(i < 5):
        model.add(layers.Conv2D(filters = 256, kernel_size = (3,3), activation = 'relu'))
    else:        
        model.add(layers.Conv2D(filters = 512, kernel_size = (3,3), activation = 'relu'))


model.add(layers.MaxPooling2D(pool_size = (2,2)))
model.add(layers.MaxPooling2D(pool_size = (2,2)))

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics = ['mean_squared_error'])

model.summary()

2023-02-02 16:59:00.272390: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-02 16:59:00.272422: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-02 16:59:00.272442: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sameep-Inspiron-5593): /proc/driver/nvidia/version does not exist
2023-02-02 16:59:00.272806: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 218, 218, 64)      36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 216, 216, 64)      36928     
                                                                 
 conv2d_4 (Conv2D)           (None, 214, 214, 64)      36928     
                                                                 
 conv2d_5 (Conv2D)           (None, 212, 212, 64)      36928     
                                                                 
 conv2d_6 (Conv2D)           (None, 210, 210, 64)      3

In [7]:
%%time

# Train the model ltered
model.fit(
    train_images,
    y_train,
    epochs = 10,
    validation_data=val_images,
    use_multiprocessing = True,
  )

ValueError: `y` argument is not supported when using `keras.utils.Sequence` as input.